# Checks nightly NRES observations

In this notebook we will check what kind of frames NRES created, i.e. how many of each calibration frames, for which fibres.

In [1]:
import os, glob
import numpy as np
from astropy.io import fits, ascii
import datetime
from astropy.table import Table, unique
from astropy.time import Time

# import request to download flat image from archive
import requests

#define the initial basename for the NRES files in archive:
basename = {   'cpt' : 'cptnrs03',
                'elp' : 'elpnrs02',
                'lsc' : 'lscnrs01',
                'tlv' : 'tlvnrs04',
            }

#define date string format for query api
fmt_api = '%Y-%m-%d'
dt1 = datetime.timedelta(days=1)

## Prepare notebook to download data from the archive

You need a userdat.dat file in your home directory or somewhere else. This file has the following two lines:
```
username=
password=
```
You have to insert user name and password after = sign.


In [2]:
# Get data from user file:
user_file = open('/home/mrabus/userdata.dat', 'r') # File with the username and password to access the archive
#read username and password
username  = (user_file.readline().split('=')[-1]).split()[0]
password  = (user_file.readline().split('=')[-1]).split()[0]
# Request token and create header to access the archive
api_token = requests.post('https://archive-api.lco.global/api-token-auth/',data = {'username': username,'password': password}).json() 
headers={'Authorization': 'Token '+api_token['token']}

Define the site and night we want to check

In [3]:
#site we want to check
site = 'cpt' 
# night we want to check as datetime object
current_night = datetime.datetime(2019,5,20)

# create start and end of night in UTC, according to site.
if site == 'cpt':
    sdate = current_night.strftime(fmt_api)     +' 11%3A30'
    edate = (current_night+dt1).strftime(fmt_api)+' 11%3A00'
if site == 'elp':
    sdate = current_night.strftime(fmt_api)     +' 18%3A30'
    edate = (current_night+dt1).strftime(fmt_api)+' 18%3A00'
if site == 'lsc':
    sdate = current_night.strftime(fmt_api)     +' 16%3A30'
    edate = (current_night+dt1).strftime(fmt_api)+' 16%3A00'
if site == 'tlv':
    sdate = current_night.strftime(fmt_api)     +' 10%3A30'
    edate = (current_night+dt1).strftime(fmt_api)+' 10%3A00'

## Query the archive:

In [4]:
# Query bias frames:
response_bias = requests.get('https://archive-api.lco.global/frames/?start='+sdate+'&end='+ \
                             edate+'&OBSTYPE=BIAS&basename='+basename[site]+'&RLEVEL=0&TELID=igla', \
                             headers=headers).json() 
# Query dark frames:
response_dark = requests.get('https://archive-api.lco.global/frames/?start='+sdate+'&end='+ \
                             edate+'&OBSTYPE=DARK&basename='+basename[site]+'&RLEVEL=0&TELID=igla', \
                             headers=headers).json() 
# Query lamp flats:
response_flat = requests.get('https://archive-api.lco.global/frames/?start='+sdate+'&end='+ \
                             edate+'&OBSTYPE=LAMPFLAT&basename='+basename[site]+'&RLEVEL=0&TELID=igla', \
                             headers=headers).json() 
# Query arcs:
response_arc = requests.get('https://archive-api.lco.global/frames/?start='+sdate+'&end='+ \
                             edate+'&OBSTYPE=DOUBLE&basename='+basename[site]+'&RLEVEL=0&TELID=igla', \
                             headers=headers).json() 
# Query target/science expousres:
response_target = requests.get('https://archive-api.lco.global/frames/?start='+sdate+'&end='+ \
                            edate+'&OBSTYPE=TARGET&basename='+basename[site]+'&RLEVEL=0&TELID=igla', \
                            headers=headers).json() 

## Target frames:

Print header information from the target fits.

In [5]:
frames = response_target['results']
for frame in frames:
    frame_id = frame['id']
    hdr = requests.get(f'https://archive-api.lco.global/frames/{frame_id}/headers/', \
                            headers=headers).json()
    print( hdr['data']['OBSTYPE'], hdr['data']['OBJECTS'], hdr['data']['MJD_OBS'], 
            Time(hdr['data']['MJD_OBS'],format='mjd').jd, hdr['data']['EXPTIME'],hdr['data']['GROUPID'],
            hdr['data']['SHUTTER0'],hdr['data']['SHUTTER1'])


## Arc frames:

Print header information from the arc fits.

In [6]:
frames = response_arc['results']
for frame in frames:
    frame_id = frame['id']
    hdr = requests.get(f'https://archive-api.lco.global/frames/{frame_id}/headers/', \
                            headers=headers).json()
    print( hdr['data']['OBSTYPE'], hdr['data']['OBJECTS'], hdr['data']['MJD_OBS'], 
            Time(hdr['data']['MJD_OBS'],format='mjd').jd, hdr['data']['EXPTIME'],hdr['data']['GROUPID'],
            hdr['data']['SHUTTER0'],hdr['data']['SHUTTER1'])


DOUBLE thar&thar&none 58623.5572874 2458624.0572874 500.0 daily arcs (Thorium Argon) open closed
DOUBLE thar&thar&none 58623.5507847 2458624.0507847 500.001 daily arcs (Thorium Argon) open closed
DOUBLE none&thar&thar 58623.5292251 2458624.0292251 500.001 daily arcs (Thorium Argon) closed open
DOUBLE none&thar&thar 58623.5227228 2458624.0227228 500.001 daily arcs (Thorium Argon) closed open
DOUBLE none&thar&thar 58623.5162198 2458624.0162198 500.001 daily arcs (Thorium Argon) closed open


## Flat frames:

Print header information from the flat fits.


In [7]:
frames = response_flat['results']
for frame in frames:
    frame_id = frame['id']
    hdr = requests.get(f'https://archive-api.lco.global/frames/{frame_id}/headers/', \
                            headers=headers).json()
    print( hdr['data']['OBSTYPE'], hdr['data']['OBJECTS'], hdr['data']['MJD_OBS'], 
            Time(hdr['data']['MJD_OBS'],format='mjd').jd, hdr['data']['EXPTIME'],hdr['data']['GROUPID'],
            hdr['data']['SHUTTER0'],hdr['data']['SHUTTER1'])


LAMPFLAT tung&tung&none 58623.5450123 2458624.0450123 240.001 daily flats (Tungsten Halogen) open closed
LAMPFLAT tung&tung&none 58623.5415118 2458624.0415118 240.001 daily flats (Tungsten Halogen) open closed
LAMPFLAT tung&tung&none 58623.5380112 2458624.0380112 239.997 daily flats (Tungsten Halogen) open closed
LAMPFLAT none&tung&tung 58623.5108878 2458624.0108878 240.001 daily flats (Tungsten Halogen) closed open
LAMPFLAT none&tung&tung 58623.5073873 2458624.0073873 240.001 daily flats (Tungsten Halogen) closed open
LAMPFLAT none&tung&tung 58623.5038985 2458624.0038985 240.001 daily flats (Tungsten Halogen) closed open
LAMPFLAT none&tung&tung 58623.5003864 2458624.0003864 240.001 daily flats (Tungsten Halogen) closed open
LAMPFLAT none&tung&tung 58623.4968975 2458623.9968975 240.001 daily flats (Tungsten Halogen) closed open
LAMPFLAT none&tung&tung 58623.4933969 2458623.9933969 240.001 daily flats (Tungsten Halogen) closed open
LAMPFLAT none&tung&tung 58623.4898964 2458623.9898964 2